# Ridge regression

In [18]:
import numpy as np
import scipy
from sklearn.metrics import mean_squared_error
from sklearn import datasets
from sklearn.linear_model import Ridge 

def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

data = datasets.load_diabetes()
X_train, y_train = data.data, data.target

In [19]:
def fit_inverse(X, y, alpha=1.0, fit_intercept=False):
    """Direct method using the inverse"""
        # this method calculate w where w = (xt . x + lambda . Id)^-1 .Xt.Y

    I = np.eye(X.shape[1])
    if(fit_intercept):
        I[0][0] = 0
    return scipy.linalg.inv(X.T@X+alpha*I)@X.T@y

In [20]:
w = fit_inverse(X_train, y_train, alpha=0.1)

sk_model = Ridge(fit_intercept=False, alpha=0.1)
sk_model.fit(X_train, y_train)

error = rel_error(sk_model.coef_, w)
assert error <= 1e-11
print("prediction error: ", error)

prediction error:  1.8759204781764696e-13


In [21]:
from scipy.linalg import solve_triangular

def fit_qr_linear_regression(X, y):
    """ QR approach for linear regression from previous assignement"""
    #  QR decomposition consists in decomposing a matrix A
    #  into two matrix Q and R where Q is a matrix orthogonal (Q^-1 = Qt ) and R is a matrix triangular superior
    
    #  therefore, A = Q . R
    #  here we decompose Xt . X
    Q,R = scipy.linalg.qr(X.T.dot(X))
    Z = Q.T.dot(X.T.dot(y))
    return solve_triangular(R,Z,lower=False)

def fit_qr(X, y, alpha=1.0, fit_intercept=False):
    """QR approach"""
    I = np.eye(X.shape[1])
    if(fit_intercept):
        I[0][0] = 0
        
    # append the sqrt(alpha) . Id to the ligne of X    
    X = np.concatenate((X,np.sqrt(alpha)*I),axis=0)
    
    # append the vecteur col 0 the ligne of Y
    y = np.concatenate((y,np.full((X.shape[1],),0)),axis=0)
    return fit_qr_linear_regression(X,y)


In [22]:
w = fit_qr(X_train, y_train, alpha=0.1)

sk_model = Ridge(fit_intercept=False, alpha=0.1)
sk_model.fit(X_train, y_train)

error = rel_error(sk_model.coef_, w)
#assert error <= 1e-11
print("prediction error: ", error)

prediction error:  1.8343088424001178e-14


In [23]:
def fit_svd(X, y, alpha=1.0, fit_intercept=False):
    """SVD approach"""
    # this method calculate w = v . (s^2 + alpha .Id)^-1 . s . ut . y
    # where u,s vh are the decomposition svd of X 
    I = np.eye(X.shape[1])
    if(fit_intercept):
        I[0][0] = 0
        
    U,S,Vt = scipy.linalg.svd(X)
    Id = S**2 + alpha*np.diag(I)
    
    # the s returned with the method below is a vector, so we need to transpose this to a matrix diagonal
    E = np.zeros((X.shape[0],X.shape[1]))
    np.fill_diagonal(E,S)

    return Vt.T@np.diag(1/Id)@E.T@U.T@y


In [24]:
w = fit_svd(X_train, y_train, alpha=0.1)

sk_model = Ridge(fit_intercept=False, alpha=0.1)
sk_model.fit(X_train, y_train)

error = rel_error(sk_model.coef_, w)
#assert error <= 1e-11
print("prediction error: ", error)

prediction error:  2.1748448821230398e-13


# Everything in a class

In [25]:
class RidgeRegression():
    def __init__(self, fit_intercept=True, method="inverse", alpha=1.0):
        self.w = 0
        self.fit_intercept = fit_intercept # bias
        self.method = method
        self.alpha = alpha
    
    def fit(self, X, y):
        if(self.fit_intercept):
            X = np.concatenate((np.ones((X.shape[0],1)),X),axis=1)
            
        if (self.method == "inverse"):
            self.w = fit_inverse(X,y,self.alpha,self.fit_intercept)
        elif (self.method == 'qr'):
            self.w = fit_qr(X,y,self.alpha,self.fit_intercept)
        elif (self.method == 'svd'):
            self.w = fit_svd(X,y,self.alpha,self.fit_intercept)   
        
    def predict(self, X):
        if(self.fit_intercept):
            b = np.ones((X.shape[0],1))
            X = np.concatenate((b,X),axis=1)  
            
        return X@self.w
def generate_data(N=100, d=20, sigma=5):
    """ Data for Ridge """
    np.random.seed(1)
    w_star = np.random.randn(d)
    X = np.random.randn(N, d)
    y = X.dot(w_star) + np.random.normal(0, sigma, size=N)
    return X, y
X_train2, y_train2 = generate_data()

model = RidgeRegression(fit_intercept=False, method="qr", alpha=0.1)
model.fit(X_train2, y_train2)

w_solution = [2.2741331962708733,-1.4638470967067754,-1.0248494680125682,-2.0920403465511344,0.19793283915844787,-1.5186692704860287,1.4772054728555917,-0.5873242037184364,0.9478891631775056,0.20512816292505345,1.251288772139991,-2.681990788073989,0.04476204682607866,-0.8659943546608414,0.6794151132231774,-0.45806886087608134,-0.11772977214105436,-1.5167038016358336,-0.7285498050097046,1.1970655855063765]
error = rel_error(model.w, w_solution)
print("prediction error: ", error)


prediction error:  7.734475198927724e-06


## without bias

In [26]:
# OTHER APPROACHES
sk_model = Ridge(fit_intercept=False, alpha=0.1)
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict(X_train)

model = RidgeRegression(fit_intercept=False, method="inverse", alpha=0.1)
model.fit(X_train, y_train)
pred = model.predict(X_train)

error = rel_error(pred, sk_pred)
assert error <= 1e-11
print("prediction error inverse: ", error)

model_qr = RidgeRegression(fit_intercept=False, method="qr", alpha=0.1)
model_qr.fit(X_train, y_train)
pred_qr = model_qr.predict(X_train)

error_qr = rel_error(pred_qr, sk_pred)
assert error_qr <= 1e-11
print("prediction error qr: ", error_qr)

model_svd = RidgeRegression(fit_intercept=False, method="svd", alpha=0.1)
model_svd.fit(X_train, y_train)
pred_svd = model_svd.predict(X_train)

error_svd = rel_error(pred_svd, sk_pred)
assert error_svd <= 1e-11
print("prediction error svd: ", error_svd)

prediction error inverse:  7.42496493809956e-14
prediction error qr:  4.2634797330840626e-14
prediction error svd:  8.171669488412137e-14


## with bias

In [27]:
# OTHER APPROACHES
sk_model = Ridge(fit_intercept=True, alpha=0.1)
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict(X_train)

model = RidgeRegression(fit_intercept=True, method="inverse", alpha=0.1)
model.fit(X_train, y_train)
pred = model.predict(X_train)

error = rel_error(pred, sk_pred)
assert error <= 1e-11
print("prediction error inverse: ", error)

model_qr = RidgeRegression(fit_intercept=True, method="qr", alpha=0.1)
model_qr.fit(X_train, y_train)
pred_qr = model_qr.predict(X_train)

error_qr = rel_error(pred_qr, sk_pred)
#assert error_qr <= 1e-11
print("prediction error qr: ", error_qr)

model_svd = RidgeRegression(fit_intercept=True, method="svd", alpha=0.1)
model_svd.fit(X_train, y_train)
pred_svd = model_svd.predict(X_train)

error_svd = rel_error(pred_svd, sk_pred)
assert error_svd <= 1e-11
print("prediction error svd: ", error_svd)

prediction error inverse:  1.0687672323449546e-15
prediction error qr:  1.1875191470499493e-15
prediction error svd:  4.600019296831778e-15


In [30]:
# Without bias
sk_model = LR(fit_intercept=False)
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict(X_train)

model = LinearRegression(fit_intercept=False)
model.fit(X_train, y_train)
pred = model.predict(X_train)

error = rel_error(pred, sk_pred)
print("prediction error: ", error)

NameError: name 'LinearRegression' is not defined

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 20 is different from 10)